In [1]:
import glob
import os 
import json
import ujson
import numpy as np
import time

In [2]:
list_logs = glob.glob("/cm/archive/anonymous_new/revise_checkpoints/Xphi35-siglip224/SMOE/665K36/analysts/entropy_nonorm/mmstar_mmmu_val/*")

In [4]:
name_benchmark = 'mmstar'
margin_data = {}
for path_folder in list_logs:
    path_data = f"{path_folder}/{name_benchmark}.json"
    try:
        with open(path_data, 'r') as f:
            data = ujson.load(f)
    except:
        continue

    # Initialize list for processed data
    new_data = []
    data_selected = {}

    # Process the data
    for sample in data['logs']:
        sample = sample['logs_metrics_vision'][0][0]  # Access the nested structure
        check = False
        # check logs moe layers in vision transformers
        for i in range(10):
            if str(i) not in sample: 
                check = True
        if check: continue
        for id_layer, value in sample.items():
            if "time_inference" in id_layer: continue
            if id_layer not in data_selected:
                data_selected[id_layer] = []
            for name_metric, val_metric in value.items():
                if "entropy_weight_topk" in name_metric: 
                    data_selected[id_layer].append(np.mean(val_metric))
                    continue
    output_dir = os.path.dirname(path_data)  # Use the same directory as input
             
    with open(output_dir + "/results.json", 'r') as f:
        data_results = ujson.load(f)
    
    # /cm/archive/anonymous_new/revise_checkpoints/Xphi35-siglip224/SMOE/665K36/revise_Full_smoe_sharev3/analysts/0717_2237_llava...mstar_llava_model_args_82420a/mme.json
    name_model = data_results['model_configs']["model_args"].split('/')[-1].split(",")[0]
    for k in data_selected.keys():
        data_selected[k] = np.mean(data_selected[k])
    margin_data[name_model] = data_selected

with open(f"router_entropy_{name_benchmark}.json", 'w') as f:
    json.dump(margin_data, f, indent=4)

In [13]:
margin_data

{'665K36': {'0': 0.03535178835201077,
  '1': 0.07737483077449724,
  '2': 0.06751945787505247,
  '3': 0.07542872965941205,
  '4': 0.08642619879404083,
  '5': 0.08280173706907469,
  '6': 0.10013823419786058,
  '7': 0.09451774462879015,
  '8': 0.09618997223535553,
  '9': 0.10094959531375207,
  '10': 0.10313921416465503,
  '11': 0.11232754998886958,
  '12': 0.10823619851151792,
  '13': 0.11448509363892177,
  '14': 0.10420365399378352,
  '15': 0.11744103628497882,
  '16': 0.13889431612286718,
  '17': 0.1542149654070381,
  '18': 0.17566491074518611,
  '19': 0.22162607888272032,
  '20': 0.23396070649754255,
  '21': 0.29222623670333997,
  '22': 0.30224923161432765,
  '23': 0.3311616037202378,
  '24': 0.3427706141487385,
  '25': 0.36822669838159344,
  '26': 0.10400550506240688,
  'mm_projector': 0.2072025761376911}}